## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [3]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSSSS'#BBBSBBBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
121
+-----------------+--------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|          coeff_1|             coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|          res_mean|    station|total_var|undefs|              vector|  year|
+-----------------+--------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|6.030601800679151|-0.37947014476416663|-0.9474111974522849|      9.4|SSSBSSSS| 37.4767|-122.2386|       SNWD|0.9390285913957866|0.9387871786762927|0.9372823677043012|0.9543662227502928|USC00047

In [4]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [5]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00047339,1962.0,-0.37947
1,USC00043387,1962.0,-0.37947
2,USC00047821,1962.0,-0.37947
3,USC00045844,1962.0,-0.37947
4,USC00040674,1962.0,-0.37947


In [6]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,USC00040674,USC00040790,USC00040850,USC00040855,USC00042369,USC00043387,USC00043925,USC00044022,USC00045118,USC00045119,...,USC00046650,USC00046926,USC00047339,USC00047668,USC00047821,USC00047916,USC00049814,USW00023244,USW00023245,USW00093217
year,,,,,,,,,,,,,,,,,,,,,
1949.0,NaN,NaN,NaN,-0.338394,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1951.0,NaN,NaN,NaN,2.565337,NaN,NaN,-0.461532,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.373770,NaN
1952.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.728006,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.170261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954.0,-0.640272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1955.0,NaN,NaN,-0.174102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.510262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957.0,-0.691361,NaN,-0.687191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.732141,NaN,NaN,NaN,NaN,-0.687191,NaN,NaN,NaN,NaN
1958.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [7]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  115.389616809
RMS removing mean-by-station=  111.496415332
RMS removing mean-by-year   =  81.6515975456


In [8]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 115.389616809
0 after removing mean by year    = 81.6515975456
0 after removing mean by stations= 80.0354198345
1 after removing mean by year    = 79.3915039541
1 after removing mean by stations= 79.1143187061
2 after removing mean by year    = 78.9885631575
2 after removing mean by stations= 78.9293601758
3 after removing mean by year    = 78.9007020762
3 after removing mean by stations= 78.886505866
4 after removing mean by year    = 78.8793237015
4 after removing mean by stations= 78.8756137104
